<div align="right">  
    Grupo 13  
    
    André Neves da Costa - A95869 
    Filipe José Silva Castro - A96156 
</div>

## Variáveis de input

+ **N** $\rightarrow$ Dimensão da grelha de células
+ **p** $\rightarrow$ Probabilidade de cada célula da borda começar viva
+ **C** $\rightarrow$ Centro do quadrado 3x3 onde vão começar as células vivas
+ **k** $\rightarrow$ Número de iterações que serão feitas

## Variáveis auxiliares

+ **state** $\rightarrow$ Dicionário que vai guardar os valores booleanos de cada célula
+ **condic** $\rightarrow$ Lista onde se vão acumular as condições da grelha inicial
+ **tran** e **tranaux** $\rightarrow$ Listas que vão acumular as condições de transição

## Predicados

O **estado inicial** é caracterizado pelo seguinte predicado:


$$ (pc = 1 \wedge \text{célula faz parte da borda} \wedge \text{entra na probabilidade p})$$
$$\vee$$
$$ (pc = 0 \wedge \text{célula faz parte da borda} \wedge \text{não entra na probabilidade p})$$
$$\vee$$
$$ (pc = 1 \wedge \text{célula não faz parte da borda} \wedge \text{célula faz parte do quadrado 3x3 cujo centro é C})$$
$$\vee$$
$$ (pc = 0 \wedge \text{célula não faz parte da borda} \wedge \text{célula não faz parte do quadrado 3x3 cujo centro é C})$$

As **transformações possíveis** são caracterizadas pelo seguinte predicado:

$$(pc = x \wedge pc' = x \wedge \text{célula faz parte da borda})$$
$$\vee$$
$$(pc = 0 \wedge pc' = 0 \wedge \text{número vizinhos} \ne 3)$$
$$\vee$$
$$(pc = 0 \wedge pc' = 1 \wedge \text{número vizinhos} = 3)$$
$$\vee$$
$$(pc = 1 \wedge pc' = 1 \wedge (\text{número vizinhos} = 2 \vee \text{número vizinhos} = 3))$$
$$\vee$$
$$(pc = 1 \wedge pc' = 0 \wedge \text{número vizinhos} \ne 2 \wedge \text{número vizinhos} \ne 3)$$

## Implementação em Python

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import BOOL
from pysmt.typing import INT
import random

N = 10
p = 0
C = (2,2)
k = 5

### `declare`
Cria as células como inteiros associando-lhes um nome, este sendo matriz{"linha"}{"coluna"}"numero da matriz" e armazena num dicionário **state** que depois é devolvido pela função.

In [ ]:
def declare(i):
    state = {}
    for m in range(N+2):
        for n in range(N+2):
            state[f'matriz{m},{n}']=Symbol(f'matriz{m},{n}'+str(i),INT)
    return state

### `init`

Verifica se o **state** que recebe é um possível estado inicial, usando para isso o *predicado do estado inicial*.

Para verificarmos a probabilidade p quando se atribui o estado às bordas é usada a função *randomint(1,100)* de python que criará aleatóriamente um número entre 1 e 100 e vai-se comparar com p $\times$ 100, por exemplo, seja p = 0.3, a probabilidade de aleatóriamente arranjar um numero entre 1 e 100 menor ou igual que 30 será 30% e caso isto seja satisfeito a célula da borda vai começar viva.

In [ ]:
def init(state):
    condic = []
    
    for i in range(C[0] - 1, C[0]+2):
        for j in range(C[1] - 1, C[1] + 2):
            condic.append(Equals(state[f'matriz{i},{j}'], Int(1)))
    
    for i in range(N+1):
        if random.randint(1,100) <= p*100:
            condic.append(Equals(state[f'matriz{i},0'], Int(1)))
        else:
            condic.append(Equals(state[f'matriz{i},0'], Int(0)))
        
    for j in range(1,N+1):
        if random.randint(1,100) <= p*100:
            condic.append(Equals(state[f'matriz0,{j}'], Int(1)))
        else:
            condic.append(Equals(state[f'matriz0,{j}'], Int(0)))
    
    for i in range(1,C[0] - 1):
        for j in range(1,N+1):
            condic.append(Equals(state[f'matriz{i},{j}'], Int(0)))
    for i in range(C[0] + 2,N+1):
        for j in range(1,N+1):
            condic.append(Equals(state[f'matriz{i},{j}'], Int(0)))
    for i in range(C[0]-1,C[0]+2):
        for j in range(1,C[1] - 1):
            condic.append(Equals(state[f'matriz{i},{j}'], Int(0)))
        for j in range(C[1]+2,N+1):
            condic.append(Equals(state[f'matriz{i},{j}'], Int(0)))
                
    return And(condic)

### `trans`

Verifica se existe uma transição possível entre as variáveis de estado **curr** e **prox**, usando para isso o *predicado das transformações possíveis*.

É calculada a **vizinhança** de cada célula armazenando os valores das células à volta e somando-os todos, isto funciona porque as células têm valores **booleanos** e 1 significa que a célula está **viva**, portanto, somando os valores de todas as células ao seu redor vai dar o numero de células vivas que tem à sua volta.

Apesar de tal não aparecer no *predicado das transformações possíveis*, `trans` também tem em conta se as células estão nos limites da grelha quando faz a contagem da vizinhança, pois caos não se coloque essas restrições vai tentar ser calculada a vizinhança percorrendo células que não existem.

In [ ]:
def trans(curr,prox):
    tran = []
    for i in range(N+1):
        for j in range(N+1):
            tranaux = []
            
            if i == 0 or j == 0:
                tranaux.append(Equals(prox[f'matriz{i},{j}'],curr[f'matriz{i},{j}']))
            
            elif i == N and j == N:
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),  
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+1)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                Or(Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+1)]),Int(3)),
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+1)]),Int(4))),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+1)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+1)]),Int(3)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+1)]),Int(4)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            elif i == N and j != N:
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),  
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+2)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                Or(Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+2)]),Int(3)),
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+2)]),Int(4))),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+2)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+2)]),Int(3)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+1) for m in range(j-1,j+2)]),Int(4)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            elif i != N and j == N:
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),  
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+1)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                Or(Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+1)]),Int(3)),
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+1)]),Int(4))),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+1)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+1)]),Int(3)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+1)]),Int(4)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            else:
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),  
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+2)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                Or(Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+2)]),Int(3)),
                                Equals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+2)]),Int(4))),
                                Equals(prox[f'matriz{i},{j}'],Int(1))))
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(0)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+2)]),Int(3)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            
            
                tranaux.append(And(Equals(curr[f'matriz{i},{j}'],Int(1)),  
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+2)]),Int(3)),
                                NotEquals(sum([curr[f'matriz{n},{m}'] for n in range(i-1,i+2) for m in range(j-1,j+2)]),Int(4)),
                                Equals(prox[f'matriz{i},{j}'],Int(0))))
            tran.append(Or(tranaux))
                
    return And(tran)

### `gera_traco`
Recebe as funções `declare`, `init` e `trans`.
Nesta função vamos trabalhar com um **solver z3** chamado **s**.

Primeiramente é criada uma lista **trace** onde se vai armazenar todas as **k** cópias de uma grelha,
é depois adicionado a **s** o `init` do primeiro elemento de **trace** e os `trans` de todos os **trace** entre **0** e **k-1**.

Finalmente, são imprimidos os resultados do **solver**, traduzindo os 0 e 1 de cada passo para ⬜ e ⬛ respetivamente, com o intuito de melhorar a leitura.

In [ ]:
def gera_traco(declare,init,trans):

    with Solver(name="z3") as s:
        
    
        # criar k cópias do estado
        trace = [declare(i) for i in range(k)]
    
        # criar o traço
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i],trace[i+1]))
        
        for i in range(k):
            for m in range(N+1):
                for n in range(N+1):
                    s.add_assertion(Or(Equals(trace[i][f'matriz{m},{n}'],Int(0)),Equals(trace[i][f'matriz{m},{n}'],Int(1))))
        
        if s.solve():
            for i in range(k):
                print("-----------------------------------------------")
                print("Passo: " + str(i) + ":")
                res=""
                for m in range(N+1):
                    res+=("\n")
                    for n in range(N+1):
                        if str(s.get_value(trace[i][f'matriz{m},{n}']))=="0":
                            res+="⬜"
                        if str(s.get_value(trace[i][f'matriz{m},{n}']))=="1":
                            res+="⬛"
                    
                print(res)
                        
                
        else:
            print("Impossivel criar um caminho")
                
gera_traco(declare,init,trans)

## Invariantes

Agora, utilizando invariantes queremos verificar se:

+ Todos os estados acessíveis contém pelo menos uma célula viva.
+ Toda a célula normal está viva pelo menos uma vez em algum estado acessível.

Para isso criamos então o invariante1 e invariante2 que se resumem a:

In [ ]:
def invariante1(state):
    return GE(sum([state[f'matriz{n},{m}'] for n in range(N+1) for m in range(N+1)]),Int(1))

In [ ]:
def invariante2(state,K):
    list=[]
    for i in range(N+1):
        for j in range(N+1):
            list.append(GE(sum([state[k][f'matriz{i},{j}'] for k in range(K)]),Int(1)))
    return And(list)

Adaptamos então o `gera_traco` de forma a conseguir receber invariantes:

In [ ]:
def gera_traco(declare,init,trans,inv1,inv2):

    with Solver(name="z3") as s:
        
    
        # criar k cópias do estado
        trace = [declare(i) for i in range(k)]
    
        # criar o traço
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i],trace[i+1]))
        
        for i in range(k):
            for m in range(N+1):
                for n in range(N+1):
                    s.add_assertion(Or(Equals(trace[i][f'matriz{m},{n}'],Int(0)),Equals(trace[i][f'matriz{m},{n}'],Int(1))))
        
        for i in range(k):
            s.add_assertion(inv1(trace[i]))
            
        s.add_assertion(Not(inv2(trace,k)))
        
        if s.solve():
            for i in range(k):
                print("-----------------------------------------------")
                print("Passo: " + str(i) + ":")
                res=""
                for m in range(N+1):
                    res+=("\n")
                    for n in range(N+1):
                        if str(s.get_value(trace[i][f'matriz{m},{n}']))=="0":
                            res+="⬜"
                        if str(s.get_value(trace[i][f'matriz{m},{n}']))=="1":
                            res+="⬛"
                    
                print(res)
                        
                
        else:
            print("Impossivel criar um caminho")

                
gera_traco(declare,init,trans,invariante1,invariante2)

## Observação

Ao usar o **invariante1** obtemos as soluções normalmente, querendo isto dizer que é uma propriedade do problema, no entanto, quando é usado o **invariante2** é quase sempre devolvido pelo programa que não é possível gerar um caminho, logo o **invariante2** não é uma propriedade do problema.

## Testes e os seus outputs

In [ ]:
"""
N = 10
p = 0.5
C = (5,5)
k = 7

-----------------------------------------------
Passo: 0:

....####......######..
......................
......................
##....................
##......######........
........######........
........######........
......................
##....................
......................
......................

-----------------------------------------------
Passo: 1:

....####......######..
................##....
......................
##........##..........
##......##..##........
......##......##......
........##..##........
..........##..........
##....................
......................
......................

-----------------------------------------------
Passo: 2:

....####......######..
..............######..
......................
##........##..........
##......######........
......####..####......
........######........
..........##..........
##....................
......................
......................

-----------------------------------------------
Passo: 3:

....####......######..
..............##..##..
................##....
##......######........
##....##......##......
......##......##......
......##......##......
........######........
##....................
......................
......................

-----------------------------------------------
Passo: 4:

....####......######..
..............##..##..
..........########....
##......########......
##....##..##..##......
....######..######....
......##..##..##......
........######........
##........##..........
......................

-----------------------------------------------------------
Passo: 5:

....####......######..
..................##..
........##............
##....................
######................
....##..........##....
....##..........##....
......................
##......######........
......................
......................

-----------------------------------------------
Passo: 6:

....####......######..
......##..........##..
......................
##....................
##..##................
....####..............
......................
..........##..........
##........##..........
..........##..........
......................

----------------------------------------------

N = 10
p = 1
C = (5,5)
k = 7

-----------------------------------------------
Passo: 0:

######################
##....................
##....................
##....................
##......######........
##......######........
##......######........
##....................
##....................
##....................
##....................

-----------------------------------------------
Passo: 1:

######################
##..################..
####..................
####......##..........
####....##..##........
####..##......##......
####....##..##........
####......##..........
####..................
####..................
##....................

-----------------------------------------------
Passo: 2:

######################
##..................##
##....##......####....
##..##....##..........
##......######........
##....####..####......
##......######........
##..##....##..........
##..##................
##....................
####..................

-----------------------------------------------
Passo: 3:

######################
##........##........##
##....................
##....##..##..##......
##............##......
####..##......##......
##............##......
##....########........
##....................
##....................
####..................

-----------------------------------------------
Passo: 4:

######################
##..####..##..####..##
####....##..##........
####........##........
##..##..##....####....
####........######....
##..####..##..##......
####....######........
####....####..........
##....................
####..................

-----------------------------------------------
Passo: 5:

######################
##..................##
##....####..##........
##..####....##........
##..##....##....##....
##......####..........
##..####........##....
##....................
##......##..##........
##....................
####..................

-----------------------------------------------
Passo: 6:

######################
##..............##..##
##..########..........
##..##......####......
##..##....####........
##..##..####..........
##....####............
##....##..............
####..................
##....................
####..................

------------------------------------------------------------------------------------

N = 10
p = 0
C = (5,5)
k = 7

-----------------------------------------------
Passo: 0:

......................
......................
......................
......................
........######........
........######........
........######........
......................
......................
......................
......................
-----------------------------------------------
Passo: 1:

......................
......................
......................
..........##..........
........##..##........
......##......##......
........##..##........
..........##..........
......................
......................
......................
-----------------------------------------------
Passo: 2:

......................
......................
......................
..........##..........
........######........
......####..####......
........######........
..........##..........
......................
......................
......................
-----------------------------------------------
Passo: 3:

......................
......................
......................
........######........
......##......##......
......##......##......
......##......##......
........######........
......................
......................
......................
-----------------------------------------------
Passo: 4:

......................
......................
..........##..........
........######........
......##..##..##......
....######..######....
......##..##..##......
........######........
..........##..........
......................
......................
-----------------------------------------------
Passo: 5:

......................
......................
........######........
......................
....##..........##....
....##..........##....
....##..........##....
......................
........######........
......................
......................
-----------------------------------------------
Passo: 6:

......................
..........##..........
..........##..........
..........##..........
......................
..######......######..
......................
..........##..........
..........##..........
..........##..........
......................



N = 10
p = 0
C = (2,2)
k = 5

-----------------------------------------------
Passo: 0:

......................
..######..............
..######..............
..######..............
......................
......................
......................
......................
......................
......................
......................
-----------------------------------------------
Passo: 1:

......................
..##..##..............
........##............
..##..##..............
....##................
......................
......................
......................
......................
......................
......................
-----------------------------------------------
Passo: 2:

......................
......................
......####............
....####..............
....##................
......................
......................
......................
......................
......................
......................
-----------------------------------------------
Passo: 3:

......................
......................
....######............
....##..##............
....####..............
......................
......................
......................
......................
......................
......................
-----------------------------------------------
Passo: 4:

......................
......##..............
....##..##............
..##....##............
....####..............
......................
......................
......................
......................
......................
......................
"""